In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [5]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

def test_da_shit(actuals, preds):
    new_test= pd.DataFrame({'Actuals': actuals,'Preds': preds})
    new_test = new_test.loc[new_test['Actuals'] != 0,:]
    return metric(np.array(new_test['Actuals']), np.array(new_test['Preds']))

def mean_encoder(df, col, target = 'Sales'):
    Mean_encoded_subject = df.groupby([col])[target].mean().to_dict() 
    return df[col].map(Mean_encoded_subject)

In [8]:
train = pd.read_csv('data/train.csv', parse_dates = True)
stores = pd.read_csv('data/store.csv')

In [20]:
full_raw = pd.merge(train, stores, how='outer', on='Store')

In [22]:
raw = full_raw.Open.loc[(full_raw.Sales == 0)] = 0
raw = full_raw.Open.loc[(full_raw.Sales > 0)] = 1

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
raw = full_raw.loc[~((full_raw.Sales.isnull()) | (full_raw.Store.isnull()))]

In [49]:
raw.Date = pd.to_datetime(raw.Date, format='%Y-%m-%d')
raw.DayOfWeek = raw.Date.dt.weekday + 1

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [50]:
raw_open = raw[~(raw.Open==0)]

In [51]:
customer_means = raw_open.groupby(['Store', 'DayOfWeek']).mean()['Customers']
customer_nulls = raw_open.loc[raw_open['Customers'].isnull()]
for i, row in customer_nulls.iterrows():
    store = row['Store']
    DayOfWeek = row['DayOfWeek']
    mean = customer_means.loc[store, DayOfWeek]
    raw_open.loc[i, 'Customers'] = mean

In [52]:
raw_open.reset_index(inplace = True, drop = True)
null_promo = raw_open.loc[raw_open['Promo'].isnull()]

for i, row in null_promo.iterrows():
    above_and_below = raw_open.loc[i-1, 'Promo'] + raw_open.loc[i+1, 'Promo']
    if above_and_below == 2:
        raw_open.loc[i, 'Promo'] = 1
    elif above_and_below == 0:
        raw_open.loc[i, 'Promo'] = 0
        
null_promo = raw_open.loc[raw_open['Promo'].isnull()]

for i, row in null_promo.iterrows():
    date = str(row['Date'])
    mean = raw_open.groupby('Date').mean().loc[date, 'Promo']
    if mean > 0.9:
        raw_open.loc[i, 'Promo'] = 1
    else:
        raw_open.loc[i, 'Promo'] = 0

In [53]:
holiday_nulls = raw_open.loc[raw_open['StateHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = raw_open.loc[raw_open['Date'] == date, 'StateHoliday'].mode()[0]
    raw_open.loc[i, 'StateHoliday'] = mode
    
holiday_nulls = raw_open.loc[raw_open['SchoolHoliday'].isnull()]
for i, row in holiday_nulls.iterrows():
    date = str(row['Date'])
    mode = raw_open.loc[raw_open['Date'] == date, 'SchoolHoliday'].mode()[0]
    raw_open.loc[i, 'SchoolHoliday'] = mode

In [105]:
clean_open_stores = raw_open.fillna(value=0)

In [106]:
# Baseline
X = clean_open_stores[['Store', 'DayOfWeek']]
y = clean_open_stores['Sales']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store', 'DayOfWeek']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = ['Store', 'DayOfWeek']).loc[:, 'Sales']
actuals = np.array(y_test)
test_da_shit(actuals, preds)

23.534890461313807

In [107]:
# Fixing the outlier (Store #817)
clean_open_stores.Customers.loc[((clean_open_stores.Sales >= 20000) & (clean_open_stores.Customers >= 7000))] = clean_open_stores.loc[(clean_open_stores.Store == 817)].mean()['Customers']

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [109]:
# Encoding Store Type 'b' vs not 'b'
clean_open_stores.loc[(clean_open_stores['StoreType'] == 'b'),'StoreType'] = 1
clean_open_stores.loc[((clean_open_stores['StoreType'] == 'a')
                      |(clean_open_stores['StoreType'] == 'c')
                      |(clean_open_stores['StoreType'] == 'd')),'StoreType'] = 0
clean_open_stores.StoreType = clean_open_stores.StoreType.astype('int64', copy=False)

In [111]:
# Encoding State Holidays
clean_open_stores.loc[((clean_open_stores['StateHoliday'] == 0) | (clean_open_stores['StateHoliday'] == 0.0)),'StateHoliday'] = 0
clean_open_stores.loc[((clean_open_stores['StateHoliday'] == 'a') | (clean_open_stores['StateHoliday'] == 'b') | (clean_open_stores['StateHoliday'] == 'c')),'StateHoliday'] = 1
clean_open_stores.StateHoliday = clean_open_stores.StateHoliday.astype('int64', copy=False)

In [112]:
# Promo2 Length Engineering
clean_open_stores['Promo2Time'] = 2015 - clean_open_stores['Promo2SinceYear']
clean_open_stores.Promo2Time.loc[(clean_open_stores.Promo2Time == 2015)] = 0

In [113]:
# Competition Time Engineering
clean_open_stores['CompetitionTime'] = 2015 - clean_open_stores['CompetitionOpenSinceYear']
clean_open_stores.CompetitionTime.loc[(clean_open_stores.CompetitionTime == 2015)] = 0
engineered_years = clean_open_stores.drop(columns = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear','Promo2SinceWeek', 'Promo2SinceYear'])

In [114]:
# Assortment Encoding
engineered_years.Assortment.value_counts()
engineered_years.loc[(engineered_years['Assortment'] == 'a'),'Assortment'] = 0
engineered_years.loc[(engineered_years['Assortment'] == 'b'),'Assortment'] = 1
engineered_years.loc[(engineered_years['Assortment'] == 'c'),'Assortment'] = 2
engineered_years.Assortment = engineered_years.Assortment.astype('int64', copy=False)

In [115]:
# Competition Distance Engineering Based on Quartiles
engineered_years.loc[((engineered_years['CompetitionDistance'] > 0) & (engineered_years['CompetitionDistance'] <= 710)),'CompetitionDistance'] = 3
engineered_years.loc[((engineered_years['CompetitionDistance'] > 710) & (engineered_years['CompetitionDistance'] <= 2320)),'CompetitionDistance'] = 2
engineered_years.loc[((engineered_years['CompetitionDistance'] > 2320) & (engineered_years['CompetitionDistance'] <= 6880)),'CompetitionDistance'] = 1
engineered_years.loc[((engineered_years['CompetitionDistance'] > 6880) & (engineered_years['CompetitionDistance'] <= 75860)),'CompetitionDistance'] = 0

In [118]:
engineered_years['MeanPurchase'] = engineered_years.Sales / engineered_years.Customers

In [122]:
final_set = engineered_years.drop(columns=['PromoInterval', 'Customers'])

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2Time,CompetitionTime,MeanPurchase
0,2013-01-02,1115.0,3,3697.0,1.0,0.0,0,1.0,0,2,1.0,1.0,3.0,0.0,12.121311
1,2013-01-03,1115.0,4,4297.0,1.0,0.0,0,1.0,0,2,1.0,1.0,3.0,0.0,14.323333
2,2013-01-04,1115.0,5,4540.0,1.0,0.0,0,1.0,0,2,1.0,1.0,3.0,0.0,13.926380
3,2013-01-05,1115.0,6,4771.0,1.0,0.0,0,1.0,0,2,1.0,1.0,3.0,0.0,14.073746
4,2013-01-07,1115.0,1,6905.0,1.0,1.0,0,1.0,0,2,1.0,1.0,3.0,0.0,14.660297


In [121]:
sample = final_set.sample(frac = 0.2, random_state = 42)

In [125]:
# Random forest
rf_set = sample.drop(['Date'], axis = 1)
X = rf_set.drop(columns = 'Sales')
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

16.054651866990916

In [124]:
# Random forest 2
rf_set = sample.drop(['Date'], axis = 1)
X = rf_set.drop(columns = 'Sales')
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
mask = y_train > 1200
X_train = X_train[mask]
y_train = y_train[mask]
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
actuals = np.array(y_test)
test_da_shit(actuals, preds)

16.099756769172103

In [ ]:
# Random forest 2
rf_set = sample.drop(['Date'], axis = 1)
X = rf_set.drop(columns = 'Sales')
y = rf_set['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

mask = y_train > 500
X_train = X_train[mask]
y_train = y_train[mask]

for i in range(40,120,5):
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators=i,random_state=42)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    actuals = np.array(y_test)
    print(f'{i}: {test_da_shit(actuals, preds)}')

40: 16.055042359396957
45: 16.047284872835032
50: 16.07244736350245
55: 16.08598137535746
60: 16.073852730506125
65: 16.059648673018142
70: 16.039740975147225
75: 16.040989824123628
80: 16.047988538601665
